In [9]:
import pandas as pd
import re
from datetime import datetime

def clean_nse_options_data(input_filename='op060825.csv', output_filename='cleaned_nifty_options.csv', target_symbol='NIFTY'):
   
    print(f"Starting data cleaning process for '{input_filename}'...")

    try:
        df = pd.read_csv(input_filename)
        print(f"Successfully loaded '{input_filename}'. Found {len(df)} total contracts.")

        # Step 1: Extract Symbol and Filter
        def extract_symbol(contract_name):
            prefix = 'OPTSTK' if contract_name.startswith('OPTSTK') else 'OPTIDX'
            prefix_len = len(prefix)
            match = re.search(r'\d', contract_name[prefix_len:])
            if match:
                end_index = match.start()
                return contract_name[prefix_len:prefix_len + end_index]
            return None

        df['SYMBOL'] = df['CONTRACT_D'].apply(extract_symbol)
        
        options_df = df[df['SYMBOL'] == target_symbol].copy()
        print(f"Filtered down to {len(options_df)} contracts for symbol '{target_symbol}'.")

        if options_df.empty:
            print(f"Error: No contracts found for symbol '{target_symbol}'. Please check the input file.")
            return

        # Step 2: Parse Contract Details 
        def parse_contract_details(contract_name):
            pattern = r'(\d{2}-[A-Z]{3}-\d{4})(CE|PE)([0-9\.]+)$'
            match = re.search(pattern, contract_name)
            if match:
                expiry_date_str = match.group(1)
                option_type = 'call' if match.group(2) == 'CE' else 'put'
                strike_price = float(match.group(3))
                return expiry_date_str, strike_price, option_type
            return None, None, None
        parsed_results = options_df['CONTRACT_D'].apply(parse_contract_details)
        options_df[['Expiry_Date_Str', 'Strike_Price', 'Option_Type']] = pd.DataFrame(parsed_results.tolist(), index=options_df.index)
        
        #Step 3: Cleaning and Formatting
        options_df.rename(columns={
            'CLOSE_PRIC': 'Market_Price',
            'UNDRLNG_ST': 'Spot_Price',
            'TRADED_QUA': 'Volume',
            'OI_NO_CON': 'Open_Interest'
        }, inplace=True)

        options_df['Expiry_Date'] = pd.to_datetime(options_df['Expiry_Date_Str'], format='%d-%b-%Y', errors='coerce')
        
        final_df = options_df[[
            'Expiry_Date', 'Strike_Price', 'Option_Type', 'Spot_Price',
            'Market_Price', 'Volume', 'Open_Interest'
        ]].copy()

        final_df.dropna(subset=['Market_Price', 'Strike_Price', 'Expiry_Date'], inplace=True)
        
        final_df = final_df[final_df['Volume'] > 0]
        
        print(f"Finished cleaning. {len(final_df)} valid, tradable contracts remaining.")

        final_df.to_csv(output_filename, index=False)
        print(f"Successfully saved cleaned data to '{output_filename}'.")

    except FileNotFoundError:
        print(f"Error: The file '{input_filename}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    clean_nse_options_data()


Starting data cleaning process for 'op060825.csv'...
Successfully loaded 'op060825.csv'. Found 28659 total contracts.
Filtered down to 1527 contracts for symbol 'NIFTY'.
Finished cleaning. 904 valid, tradable contracts remaining.
Successfully saved cleaned data to 'cleaned_nifty_options.csv'.
